# TimesFM スモークテスト（テンプレ）

このノートブックは、あなたの PostgreSQL（PostgresSQL：オープンソースRDB）上の  
`dataset.loto_y_ts` と `dataset.loto_hist_feat` を使って `timesfm` を段階的に疎通確認するための **テンプレ** です。

> 注意: この実行環境に `timesfm` が入っていない場合があります。  
> その場合は、あなたの環境（/mnt/e/env/ts/lib_ana/…）でこのノートブックを使ってください。

## 目的（最短ルート）
1. DBから取得 → DataFrame（データフレーム：表形式データ）整形
2. `forecast_on_df()` で単変量の疎通
3. `forecast_with_covariates()` で共変量（covariates：説明変数）併用の疎通


In [ ]:
# === 接続情報（あなたの設定に合わせる） ===
DB_HOST = "127.0.0.1"
DB_PORT = 5432
DB_USER = "loto"
DB_NAME = "loto"
DB_SCHEMA = "dataset"


In [ ]:
# === DBから読み出し（Pythonで直接） ===
# 依存: psycopg2 または SQLAlchemy が必要です。
# もし無ければ、psql の \copy でCSVを吐いて pandas.read_csv でもOK。

import pandas as pd

try:
    import psycopg2  # type: ignore
except Exception as e:
    psycopg2 = None
    print("psycopg2 がありません。psql の \copy でCSVを作る方式を推奨:", e)

def read_table_sql(table: str, where: str = "") -> pd.DataFrame:
    if psycopg2 is None:
        raise RuntimeError("psycopg2 が無いため Python 直結は不可。psql \copy を使ってください。")
    sql = f"SELECT * FROM {DB_SCHEMA}.{table} " + (where if where else "")
    conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, user=DB_USER, dbname=DB_NAME)
    try:
        df = pd.read_sql(sql, conn)
    finally:
        conn.close()
    return df

# 例: 直近365日
# df_y = read_table_sql("loto_y_ts", "WHERE ds >= CURRENT_DATE - INTERVAL '365 days' ORDER BY unique_id, ds")
# df_x = read_table_sql("loto_hist_feat", "WHERE ds >= CURRENT_DATE - INTERVAL '365 days' ORDER BY unique_id, ds")


In [ ]:
# === psql \copy でCSVを作る場合のコマンド例（手動で実行） ===
# mkdir -p /mnt/e/env/ts/lib_ana/data
# psql -h 127.0.0.1 -p 5432 -U loto -d loto -v ON_ERROR_STOP=1 -c "\
# \copy (SELECT loto, unique_id, ds, ts_type, y
#        FROM dataset.loto_y_ts
#        WHERE ds >= CURRENT_DATE - INTERVAL '365 days'
#        ORDER BY unique_id, ds) TO '/mnt/e/env/ts/lib_ana/data/loto_y_ts_smoke.csv' WITH (FORMAT csv, HEADER true)"
#
# psql -h 127.0.0.1 -p 5432 -U loto -d loto -v ON_ERROR_STOP=1 -c "\
# \copy (SELECT *
#        FROM dataset.loto_hist_feat
#        WHERE ds >= CURRENT_DATE - INTERVAL '365 days'
#        ORDER BY unique_id, ds) TO '/mnt/e/env/ts/lib_ana/data/loto_hist_feat_smoke.csv' WITH (FORMAT csv, HEADER true)"


In [ ]:
# === データ整形（最低限） ===
# df_y / df_x を用意した前提

def prep_y(df_y: pd.DataFrame) -> pd.DataFrame:
    df = df_y.copy()
    df["ds"] = pd.to_datetime(df["ds"])
    # TimesFM の一般的な期待列: unique_id, ds, y
    return df[["unique_id", "ds", "y"]].sort_values(["unique_id", "ds"]).dropna()

def prep_x(df_x: pd.DataFrame) -> pd.DataFrame:
    df = df_x.copy()
    df["ds"] = pd.to_datetime(df["ds"])
    # 共変量候補（例: hist_*）
    hist_cols = [c for c in df.columns if c.startswith("hist_")]
    keep = ["unique_id", "ds"] + hist_cols
    return df[keep].sort_values(["unique_id", "ds"])


In [ ]:
# === TimesFM 疎通（timesfm がインストール済みの場合） ===
# ここはあなたの環境で checkpoint と hparams を設定して実行してください。

try:
    import timesfm  # type: ignore
    from timesfm.timesfm_base import TimesFmHparams, TimesFmCheckpoint  # type: ignore
except Exception as e:
    timesfm = None
    print("timesfm が import できません:", e)

# 例（※値は仮。checkpointに合わせて要調整）
# hparams = TimesFmHparams(
#     context_len=512,
#     horizon_len=128,
#     input_patch_len=32,
#     output_patch_len=32,
#     num_layers=20,
#     model_dims=1280,
#     per_core_batch_size=32,
#     backend="cpu",   # "gpu" 等
#     quantiles=[0.1, 0.5, 0.9],
# )
#
# ckpt = TimesFmCheckpoint(version="torch", path="/path/to/checkpoint")
#
# model = timesfm.TimesFmTorch(hparams)  # or TimesFmJax(...)
# model.load_from_checkpoint(ckpt)
#
# pred_df = model.forecast_on_df(df=prep_y(df_y), freq="D", value_name="y")
# pred_df.head()
